# 早期終了 (Early stopping)

In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchinfo
from tqdm import tqdm
import utils

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.3の大きさのガウシアンノイズがのったデータを考えます。データ数は30にします。

In [ ]:
# データ点の取得
x, t = utils.dataset_overtraining(n=30, noise_scale=0.3)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x, t, s=10, c="black", label="data")  # データ点のプロット
x_grid = np.linspace(-np.pi, np.pi, 100)
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

NumPy arrayをPyTorch tensorに変換します。

In [ ]:
# numpy -> torch.tensor
x_tensor = torch.from_numpy(x).float().unsqueeze(-1)
t_tensor = torch.from_numpy(t).float().unsqueeze(-1)
x_grid_tensor = torch.from_numpy(x_grid).float().unsqueeze(-1)

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、隠れ層が5層、ノード数が128の隠れ層を4層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

# 誤差関数として二乗和誤差を指定。最適化手法はAdam
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# トレーニング
num_epochs = 1000
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x, t, s=10, c="black", label="data")  # データ点のプロット
ax.plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()


モデルの出力(赤線)がトレーニングデータ点に強くフィットしてしまっていることがわかります。

## データの分割
Early stoppingでは データセットをランダムに2つに分割します。
一方のデータセット(training dataset)でトレーニングをし、もう一方のデータセット(validation dataset)で過学習がおこらないかを確認します。
サンプルの分割は手で行うこともできますが、scikit learnというライブラリに便利な関数が用意されているので、これを用いることにします。

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.30, random_state=0)

# numpy -> torch.tensor
x_train_tensor = torch.from_numpy(x_train).float().unsqueeze(-1)
t_train_tensor = torch.from_numpy(t_train).float().unsqueeze(-1)
x_valid_tensor = torch.from_numpy(x_valid).float().unsqueeze(-1)
t_valid_tensor = torch.from_numpy(t_valid).float().unsqueeze(-1)

上の例では、全体の70%をトレーニングデータ、全体の30%を検証用データ(validation sample)として分割しています。

トレーニングデータ、検証用データをプロットすると下図のようになります。
黒点がトレーニングデータ、オレンジ点が検証用データです。
全データセットからランダムに分割されていることがわかります。

In [ ]:
# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x_train, t_train, s=10, c="black", label="training data")
ax.scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

検証用データに対して監視をするために、エポックごとに検証用データのロスを計算するコードを実装します。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

# 誤差関数として二乗和誤差を指定。最適化手法はAdam
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

結果をプロットしてみます。

In [ ]:
# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

学習に使った黒点に沿うようにモデルがフィットされていることがわかります。
一方で、検証用の点(オレンジ色の点)に対しての予測値は大きくずれてしまっています。

誤差関数の値(loos)の推移を見ると、epoch数が およそ80 あたりで、検証用データの誤差関数の値が下がらなくなってしまってしまい、学習を進めるにつれ、検証用データに対する誤差はむしろ大きくなってしまっています。
これが過学習を起こした状態で、トレーニングデータに強く学習してしまったため、本来の目的である汎化誤差が大きくなってしまっています。

これは、検証用データの誤差関数値がそれ以上下がらなくなった点で学習を止めることで緩和することができます。
上の例だとおよそ80 エポックぐらいで誤差関数値が上昇に転じているので、このあたりで学習を止めることにしましょう。

In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

# 誤差関数として二乗和誤差を指定。最適化手法はAdam
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 80
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

今度はどうなったでしょうか？おそらく、過学習が緩和されたのではないかと思います。

このように、学習を途中で打ち切るような手法を早期終了(Early Stopping)と言います。

validationロスの値をモニターして、一定期間ロスの値が改善しない場合に学習を止めるように学習を工夫してみます。


In [ ]:
# モデルの定義
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1),
)

# 誤差関数として二乗和誤差を指定。最適化手法はAdam
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

loss_best = torch.finfo(torch.float).max  # ロスの最小値を記録します
count_patience = 0  # ロスが最小となったエポックからの経過エポック数をカウントします
max_patience = 20  # ロスが最小となった後、何エポックまでトレーニングを続けるかの閾値

from tempfile import NamedTemporaryFile
# 最小ロスのときのモデルパラメータを保存するpathです。一時ファイル用にランダムなパスを生成します。
save_path = NamedTemporaryFile()

# トレーニング
num_epochs = 500
for i_epoch in tqdm(range(num_epochs)):
    # モデルをトレーニングモードにする。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # モデルを評価モードにする。
    model.eval()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor).detach().numpy()

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid]

    # Early stopping
    if loss_valid < loss_best:
        # validation ロスの値がこれまでの最小値よりも小さくなった時はloss_bestの値を更新します
        loss_best = loss_valid
        count_patience = 0

        # 後で再現できるようにモデルパラメータを保存しておきます。
        torch.save(model.state_dict(), save_path.name)
    else:
        # ロスが最小値よりも大きいときは、経過エポック数をカウントアップします。
        count_patience += 1

        # 経過エポック数が閾値より大きい時、学習を停止します。
        if count_patience >= max_patience:
            print(f"best epoch = {i_epoch - count_patience}")

            # 保存しておいたモデルパラメータを読み込みます。
            model.load_state_dict(torch.load(save_path.name))

            # モデルパラメータの一時ファイルを削除します。
            save_path.close()
            break

# モデルを評価モードにする。
model.eval()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid_tensor).detach().numpy(), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train_history, label="loss (train)")
ax[1].plot(loss_valid_history, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

`max_patience`というのは、モニターしたい値(ここでは検証用データのロス)が何エポック下がらなくなったら学習を止めるか、を指定します。今はちょっと大きめの値の20としておきます。
この値は問題・モデルによって調節するべき値です。小さすぎると、過学習を起こす前・学習が十分に行われていない段階で学習がストップしてしまいます。一方で、大きすぎると学習時間が余計にかかってしまいます。

指定した最大エポック数(500エポック)の前に学習が終了していると思います。また、過学習も緩和されています。